In [1]:
import numpy as np
from sklearn.decomposition import PCA
import pylab as plt
from sklearn.neural_network import MLPClassifier

In [2]:
# Se cargan los datos
y_total = np.genfromtxt("msd_genre_dataset.txt",dtype =None,usecols=0,comments=None,delimiter=',',skip_header=10)
x_total = np.genfromtxt("msd_genre_dataset.txt",comments=None,delimiter=',',skip_header=10)[:,4:]

generos = [b'metal',b'punk',b'dance and electronica']

y_raw = []
x_raw = []
for i in range(0,y_total.size):
    if y_total[i] in generos:
        y_raw.append(y_total[i])
        x_raw.append(x_total[i])
        
y_raw = np.asarray(y_raw)
x_raw = np.asarray(x_raw)

In [3]:
# Preprocesamiento
x_normalizado = (x_raw - np.mean(x_raw,axis=0))/np.std(x_raw,axis=0)

# PCA
proporcion_varianza = 0
n_componentes = 0
while proporcion_varianza < 0.90:
    n_componentes = n_componentes + 1
    pca = PCA(n_components=n_componentes)
    pca = pca.fit(x_normalizado)
    proporcion_varianza = np.sum(pca.explained_variance_ratio_)
    
suma_acumulada_varianza = np.cumsum(pca.explained_variance_ratio_*100)

x_preprocesamiento = pca.fit_transform(x_normalizado)
xy_preprocesamiento = np.append(x_preprocesamiento,np.array([y_raw]).T,axis=1)

# Separacion de datos para prueba y entrenamiento
proporcion_prueba_f = 0.2
proporcion_prueba_t = 0.1
np.random.shuffle(xy_preprocesamiento)
n_prueba_f = round(proporcion_prueba_f*y_raw.size)
n_prueba_t = round(proporcion_prueba_t*y_raw.size)
n_entrenamiento = y_raw.size - n_prueba_f - n_prueba_t
datos_prueba_f = xy_preprocesamiento[0:n_prueba_f,:]
x_prueba_f = datos_prueba_f[:,0:-1].astype(np.float64)
y_prueba_f = datos_prueba_f[:,-1]
datos_prueba_t = xy_preprocesamiento[n_prueba_f:n_prueba_f+n_prueba_t,:]
x_prueba_t = datos_prueba_t[:,0:-1].astype(np.float64)
y_prueba_t = datos_prueba_t[:,-1]
datos_entrenamiento = xy_preprocesamiento[n_prueba_f+n_prueba_t:,:]
x_entrenamiento = datos_entrenamiento[:,0:-1].astype(np.float64)
y_entrenamiento = datos_entrenamiento[:,-1]
etiquetas_prueba_f = np.where(y_prueba_f == generos[2], np.ones(n_prueba_f), -np.ones(n_prueba_f))
etiquetas_prueba_t = np.where(y_prueba_t == generos[2], np.ones(n_prueba_t), -np.ones(n_prueba_t))
etiquetas_entrenamiento = np.where(y_entrenamiento == generos[2], np.ones(n_entrenamiento), -np.ones(n_entrenamiento))

In [28]:
# Generacion del clasificador
puntajes = []
clasificadores_2L_NN = []
promedio_coefs = []
n_n_min = 2
n_n_max = 100
for n_neuronas in range(n_n_min,n_n_max+1):
    clasificador = MLPClassifier(solver='sgd',activation='tanh',alpha=1e-5,
                                       hidden_layer_sizes=(n_neuronas,),batch_size=10)
    clasificadores_2L_NN.append(clasificador.fit(x_entrenamiento, etiquetas_entrenamiento))
    puntajes.append(clasificador.score(x_prueba_t, etiquetas_prueba_t))
    promedio_coefs.append(np.average(clasificador.coefs_[0],axis=1))

In [31]:
plt.plot(puntajes)
plt.savefig('test_3.pdf')
plt.close()

In [32]:
a = np.asarray(promedio_coefs)
for i in range(0,n_n_max-n_n_min):
    plt.plot(np.abs(a[:,i]))
plt.savefig('test_coef_3.pdf')
plt.close()